# Computational Illustration for Chapter 15
# The Complexity of Sparse Recovery

In [1]:
import numpy as np
from numpy import linalg as LA
from scipy import linalg
from scipy.optimize import linprog

In [2]:
## generate a 2s-sparse vector and its first 2s Fourier observations
# select problem sizes
N = 500
s = 10
m = 2*s
# create the sparse vector x to be recovered
x = np.zeros(N)
aux = np.random.permutation(N)
supp_ori = aux[:s]
supp_ori.sort()
x[supp_ori] = np.random.normal(0,1,s)
# produce the observation vector y made of 2s Fourier coefficients 
xhat = np.fft.fft(x)
y_exact = xhat[:m]
# as well as a noisy version
noise = 1e-5*np.random.normal(0,1,m)
y_noisy = y_exact+noise

## Sparse recovery via Prony's method seems to be successful...

In [3]:
phat = np.zeros(N, dtype=np.cfloat)
phat[0] = 1
M = linalg.toeplitz(y_exact[s-1:2*s-1], y_exact[s-1::-1])
phat[1:s+1] = -linalg.solve(M,y_exact[s:2*s])
p = np.fft.ifft(phat)
idx = np.argsort(abs(p))
supp_exact = idx[0:s]
supp_exact.sort()
print('In the exact case, the original and recovered supports agree:')
print(supp_ori)
print(supp_exact)

In the exact case, the original and recovered supports agree:
[ 10  85 145 182 186 192 303 398 403 466]
[ 10  85 145 182 186 192 303 398 403 466]


## But it is not robust to observation errors with only m=2s

In [4]:
phat = np.zeros(N, dtype=np.cfloat)
phat[0] = 1
M = linalg.toeplitz(y_noisy[s-1:2*s-1], y_noisy[s-1::-1])
phat[1:s+1] = -linalg.solve(M,y_noisy[s:2*s])
p = np.fft.ifft(phat)
idx = np.argsort(abs(p))
supp_noisy = idx[0:s]
supp_noisy.sort()
print('In the noisy case, the original and recovered supports do not agree anymore:')
print(supp_ori)
print(supp_noisy)

In the noisy case, the original and recovered supports do not agree anymore:
[ 10  85 145 182 186 192 303 398 403 466]
[ 10  85 145 184 185 186 303 398 403 466]


In [5]:
## The outputted vectors do not agree either
F = linalg.dft(N)    # the full discrete Fourier matrix
A = F[:2*s,:]        # the submatrix for the first 2s Fourier coefficients
x_exact = np.zeros(N, dtype=np.cfloat)
x_exact[supp_exact] = linalg.solve(A[:s,supp_exact],y_exact[:s])
rel_error_exact = LA.norm(x-x_exact)/LA.norm(x)
x_noisy = np.zeros(N, dtype=np.cfloat)
x_noisy[supp_noisy] = linalg.solve(A[:s,supp_noisy],y_noisy[:s])
rel_error_noisy = LA.norm(x-x_noisy)/LA.norm(x)
print('Recovery from exact observations is quite successful: relative L2-error = {}.'.format(rel_error_exact))
print('Recovery from inexact observations is not successful: relative L2-error = {}.'.format(rel_error_noisy))

Recovery from exact observations is quite successful: relative L2-error = 2.400449957121822e-12.
Recovery from inexact observations is not successful: relative L2-error = 4.00691215658622.


## In contrast, recovery via L1-minimization is stable (with more observations, of course)

In [6]:
m = 4*s
A = F[:m,:] 
y_exact = A@x
y_noisy = y_exact+1e-5*np.random.normal(0,1,m)
obj= np.ones(2*N)
lhs_eq = np.append(A, -A, axis=1)
bnd = [(0, np.inf) for _ in range(2*N)]
opt_exact = linprog(c=obj, A_eq=lhs_eq, b_eq=y_exact, bounds=bnd, method="interior-point")
xstar_exact = opt_exact.x[:N]-opt_exact.x[N:]
rel_error_exact = LA.norm(x-xstar_exact)/LA.norm(x)   
opt_noisy = linprog(c=obj, A_eq=lhs_eq, b_eq=y_noisy, bounds=bnd, method="interior-point")
xstar_noisy = opt_noisy.x[:N]-opt_noisy.x[N:]
rel_error_noisy = LA.norm(x-xstar_noisy)/LA.norm(x)    
print('Recovery from exact observations is quite successful: relative L2-error = {}.'.format(rel_error_exact))
print('Recovery from inexact observations is not bad either: relative L2-error = {}.'.format(rel_error_noisy))

Recovery from exact observations is quite successful: relative L2-error = 0.8155465221254092.
Recovery from inexact observations is not bad either: relative L2-error = 0.8155447062531715.


/anaconda3/lib/python3.7/site-packages/numpy/core/numeric.py:538: ComplexWarning: Casting complex values to real discards the imaginary part
  return array(a, dtype, copy=False, order=order)
